In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import os
import torch
from kan import *
from kan.utils import create_dataset_from_data, ex_round

In [2]:
import pickle
import ast
from pathlib import Path

In [2]:
# Список функций
def f1(x):
    """f(x) = sin(pi * x) + x^2"""
    return np.sin(np.pi * x) + x**2

def f2(x):
    """f(x) = exp(-x^2)"""
    return np.exp(-x**2)

def f3(x):
    """f(x) = x * sin(5 * x)"""
    return x * np.sin(5 * x)

def f4(x):
    """f(x, y) = sin(pi * x) + y^2"""
    return np.sin(np.pi * x[:, 0]) + x[:, 1]**2

def f5(x):
    """f(x, y) = exp(sin(pi * x) + y^2) — как в hellokan"""
    return np.exp(np.sin(np.pi * x[:, 0]) + x[:, 1]**2)

FUNCTIONS = [
    (f1, 1, "sin(pi*x) + x^2", "D1N1"),
    (f2, 1, "exp(-x^2)", "D1N2"),
    (f3, 1, "x * sin(5*x)", "D1N3"),
    (f4, 2, "sin(pi*x) + y^2", "D2N1"),
    (f5, 2, "exp(sin(pi*x) + y^2)", "D2N1"),
]

In [ ]:
base_name = f"data_{clean_id}_noise{noise_level}_{gap_str}_n{n_samples}"

In [126]:
def stats( model, dataset, func, n_var, x_true ):
    # Предсказание модели на новых данных (например, test_input)
    x_true_clear = x_true.copy()
    if n_var==2:
        x_true_clear = np.column_stack([x_true, x_true])
    
    y_true_clear = func(x_true_clear)
    
    with torch.no_grad():
        y_pred = model(dataset['test_input'])  # shape: (N_test, 1)
        y_true = dataset['test_label']         # shape: (N_test, 1)
        y_true_pred = model( torch.tensor(x_true_clear, dtype=torch.float32, device='cpu').reshape( (len(x_true), n_var) ) ) 
    # Преобразуем в numpy для удобства (опционально)
    y_pred_np = y_pred.cpu().numpy().flatten()
    y_true_np = y_true.cpu().numpy().flatten()
    y_true_pred = y_true_pred.cpu().numpy().flatten()
    # Остатки (residuals)
    residuals = y_true_np - y_pred_np
    # MSE — среднеквадратичная ошибка
    mse = np.mean(residuals ** 2)
    # MAE — средняя абсолютная ошибка
    mae = np.mean(np.abs(residuals))
    ss_res_test = np.sum(residuals ** 2)
    ss_tot_test = np.sum((y_true_np - np.mean(y_true_np)) ** 2)
    r2 = 1 - (ss_res_test / ss_tot_test)
    

    residuals_true = y_true_clear - y_true_pred
    # MSE — среднеквадратичная ошибка
    mse_true = np.mean(residuals_true ** 2)
    # MAE — средняя абсолютная ошибка
    mae_true = np.mean(np.abs(residuals_true))
    ss_res_true = np.sum(residuals_true ** 2)
    ss_tot_true = np.sum((y_true_clear - np.mean(y_true_clear)) ** 2)
    r2_true = 1 - (ss_res_true / ss_tot_true)

    # return residuals, mse, mae, residuals_true, mse_true, mae_true
    return {
            'test_residuals': residuals,
            'test_mse': mse,
            'test_mae': mae,
            'test_r2': r2,
            'true_residuals': residuals_true,
            'true_mse': mse_true,
            'true_mae': mae_true,
            'true_r2': r2_true
        }

In [ ]:
base_cols = [
    'func_id', 'noise_level', 'gap_str', 'n_samples',
    'width', 'k', 'grid', 'seed', 'formula'  # добавили seed
]
stages = ['before_prune', 'after_prune', 'after_refine', 'after_symbolic']
loss_cols = []
metric_cols = []
for stage in stages:
    loss_cols += [
        f'loss_train_history_{stage}',
        f'loss_test_history_{stage}',
        f'reg_history_{stage}',
    ]
    metric_cols += [
        f'test_residuals_{stage}',
        f'test_mse_{stage}',
        f'test_mae_{stage}',
        f'test_r2_{stage}',
        f'true_residuals_{stage}',
        f'true_mse_{stage}',
        f'true_mae_{stage}',
        f'true_r2_{stage}',
    ]
all_columns = base_cols + loss_cols + metric_cols

{'func_id': [],
 'noise_level': [],
 'gap_str': [],
 'n_samples': [],
 'width': [],
 'k': [],
 'grid': [],
 'seed': [],
 'formula': [],
 'loss_train_history_before_prune': [],
 'loss_test_history_before_prune': [],
 'reg_history_before_prune': [],
 'loss_train_history_after_prune': [],
 'loss_test_history_after_prune': [],
 'reg_history_after_prune': [],
 'loss_train_history_after_refine': [],
 'loss_test_history_after_refine': [],
 'reg_history_after_refine': [],
 'loss_train_history_after_symbolic': [],
 'loss_test_history_after_symbolic': [],
 'reg_history_after_symbolic': [],
 'test_residuals_before_prune': [],
 'test_mse_before_prune': [],
 'test_mae_before_prune': [],
 'test_r2_before_prune': [],
 'true_residuals_before_prune': [],
 'true_mse_before_prune': [],
 'true_mae_before_prune': [],
 'true_r2_before_prune': [],
 'test_residuals_after_prune': [],
 'test_mse_after_prune': [],
 'test_mae_after_prune': [],
 'test_r2_after_prune': [],
 'true_residuals_after_prune': [],
 'true_

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 53 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   func_id                            10 non-null     object 
 1   noise_level                        10 non-null     int64  
 2   gap_str                            10 non-null     object 
 3   n_samples                          10 non-null     int64  
 4   width                              10 non-null     object 
 5   k                                  10 non-null     int64  
 6   grid                               10 non-null     int64  
 7   seed                               10 non-null     int64  
 8   formula                            10 non-null     object 
 9   loss_train_history_before_prune    10 non-null     object 
 10  loss_test_history_before_prune     10 non-null     object 
 11  reg_history_before_prune           10 non-null     object 
 1

In [5]:
from pathlib import Path

# Ваш исходный код до обработки
folder = Path('./results')
files = [f.name for f in folder.iterdir() if f.is_file()]
print(files)
pkl_files = [f for f in files if f.endswith('.pkl')]
print("Найденные .pkl файлы:", pkl_files)
pq_files = [f for f in files if f.endswith('.parquet')]
print("Найденные .parquet файлы:", pq_files)

['.gap', 'results_D1N1.csv', 'results_D1N1.parquet', 'results_D1N1.pkl', 'results_D1N1_1.csv', 'results_D1N1_1.pkl', 'results_D1N1_2.csv', 'results_D1N1_2.pkl', 'results_D1N2.csv', 'results_D1N2.parquet', 'results_D1N2.pkl', 'results_D1N2_1.csv', 'results_D1N2_1.pkl', 'results_D1N2_2.csv', 'results_D1N2_2.pkl', 'results_D1N3.csv', 'results_D1N3.parquet', 'results_D1N3.pkl', 'results_D1N3_1.csv', 'results_D1N3_1.pkl', 'results_D1N3_2.csv', 'results_D1N3_2.pkl', 'results_D2N1.csv', 'results_D2N1.parquet', 'results_D2N1.pkl', 'results_D2N1_1.csv', 'results_D2N1_1.pkl', 'results_D2N1_2.csv', 'results_D2N1_2.pkl', 'results_D2N2.csv', 'results_D2N2.parquet', 'results_D2N2.pkl', 'results_D2N2_1.csv', 'results_D2N2_1.pkl', 'results_D2N2_2.csv', 'results_D2N2_2.pkl']
Найденные .pkl файлы: ['results_D1N1.pkl', 'results_D1N1_1.pkl', 'results_D1N1_2.pkl', 'results_D1N2.pkl', 'results_D1N2_1.pkl', 'results_D1N2_2.pkl', 'results_D1N3.pkl', 'results_D1N3_1.pkl', 'results_D1N3_2.pkl', 'results_D2N1.pk

In [21]:
data_list = []
for file in pq_files:
    data_list.append(pd.read_parquet("results/"+file))

data = pd.concat(data_list, axis=0, ignore_index=True)

# Обработка столбцов loss_cols с сохранением NaN
for loss in loss_cols:
    def convert_cell(x):
        # Если значение уже NaN (или None) — оставляем как есть
        if pd.isna(x):
            return x  # или return np.nan — эквивалентно
        
        # Если это строка — пытаемся разобрать как Python-литерал
        if isinstance(x, str):
            try:
                return ast.literal_eval(x.strip())
            except (SyntaxError, ValueError):
                # Если строка невалидна — возвращаем NaN
                return np.nan
        # Если не строка и не NaN — возвращаем как есть (на всякий случай)
        return x

    data[loss] = data[loss].apply(convert_cell)

# Проверка результата
print(data[loss_cols].head())

                     loss_train_history_before_prune  \
0  [0.22302645444869995, 0.0923803374171257, 0.06...   
1  [0.13159415125846863, 0.08717337995767593, 0.0...   
2  [0.667601466178894, 0.3491753041744232, 0.2004...   
3  [0.5994330644607544, 0.15937338769435883, 0.13...   
4  [0.13066960871219635, 0.08464369922876358, 0.0...   

                      loss_test_history_before_prune  \
0  [0.25927072763442993, 0.08735374361276627, 0.0...   
1  [0.1098247841000557, 0.08423811197280884, 0.07...   
2  [0.9904837608337402, 0.43852075934410095, 0.33...   
3  [0.5983025431632996, 0.6314916014671326, 0.657...   
4  [0.1722496598958969, 0.11685020476579666, 0.11...   

                            reg_history_before_prune  \
0  [13.876544952392578, 14.417081832885742, 14.50...   
1  [14.675263404846191, 15.153724670410156, 15.15...   
2  [36.047630310058594, 37.12757873535156, 38.713...   
3  [27.257980346679688, 29.80587387084961, 28.769...   
4  [13.470575332641602, 13.265196800231934, 13

In [6]:
data_list = []
for file in pkl_files:
    with open("results/"+file, "rb") as f:
        rows = pickle.load(f)
    data_list.append(pd.DataFrame(rows))

data = pd.concat(data_list, axis=0, ignore_index=True)

# Обработка столбцов loss_cols с сохранением NaN
for loss in loss_cols:
    def convert_cell(x):
        # Если значение уже NaN (или None) — оставляем как есть
        if pd.isna(x):
            return x  # или return np.nan — эквивалентно
        
        # Если это строка — пытаемся разобрать как Python-литерал
        if isinstance(x, str):
            try:
                return ast.literal_eval(x.strip())
            except (SyntaxError, ValueError):
                # Если строка невалидна — возвращаем NaN
                return np.nan
        # Если не строка и не NaN — возвращаем как есть (на всякий случай)
        return x

    data[loss] = data[loss].apply(convert_cell)

# Проверка результата
print(data[loss_cols].head())

                     loss_train_history_before_prune  \
0  [0.22302645444869995, 0.0923803374171257, 0.06...   
1  [0.13159415125846863, 0.08717337995767593, 0.0...   
2  [0.667601466178894, 0.3491753041744232, 0.2004...   
3  [0.5994330644607544, 0.15937338769435883, 0.13...   
4  [0.13066960871219635, 0.08464369922876358, 0.0...   

                      loss_test_history_before_prune  \
0  [0.25927072763442993, 0.08735374361276627, 0.0...   
1  [0.1098247841000557, 0.08423811197280884, 0.07...   
2  [0.9904837608337402, 0.43852075934410095, 0.33...   
3  [0.5983025431632996, 0.6314916014671326, 0.657...   
4  [0.1722496598958969, 0.11685020476579666, 0.11...   

                            reg_history_before_prune  \
0  [13.876544952392578, 14.417081832885742, 14.50...   
1  [14.675263404846191, 15.153724670410156, 15.15...   
2  [36.047630310058594, 37.12757873535156, 38.713...   
3  [27.257980346679688, 29.80587387084961, 28.769...   
4  [13.470575332641602, 13.265196800231934, 13

In [7]:
data

,func_id,noise_level,gap_str,n_samples,width,k,grid,seed,formula,loss_train_history_before_prune,...,true_mae_after_refine,true_r2_after_refine,test_residuals_after_symbolic,test_mse_after_symbolic,test_mae_after_symbolic,test_r2_after_symbolic,true_residuals_after_symbolic,true_mse_after_symbolic,true_mae_after_symbolic,true_r2_after_symbolic
0,D1N1,0.00,nogap,100,"[1, 5, 1]",1,3,600076812,1.4882 - 0.3372*x_1,"[0.22302645444869995, 0.0923803374171257, 0.06...",...,0.147888,0.896281,"[-2.4055583, 1.7071576, 1.7983644, -1.7691612,...",2.020545,1.189563,0.109174,"[1.837373971939087, 1.8332574976040399, 1.8292...",1.892972,1.078047,0.020519
1,D1N1,0.00,nogap,100,"[1, 5, 1]",1,3,1002779995,nan,"[0.13159415125846863, 0.08717337995767593, 0.0...",...,0.151268,0.874985,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN
2,D1N1,0.00,nogap,100,"[1, 4, 4, 4, 1]",1,3,785117571,nan,"[0.667601466178894, 0.3491753041744232, 0.2004...",...,0.139006,0.944912,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN
3,D1N1,0.00,nogap,100,"[1, 4, 4, 4, 1]",1,3,41416371,-0.4058*x_1 - 5.504*sin(1.1431*x_1 - 4.4575) +...,"[0.5994330644607544, 0.15937338769435883, 0.13...",...,0.229275,0.827259,"[-0.0015078187, -0.0035214424, 0.0023185015, 0...",0.000008,0.002594,0.999997,"[0.007747650146484375, 0.006915391738561372, 0...",0.000008,0.002047,0.999996
4,D1N1,0.05,nogap,100,"[1, 5, 1]",1,3,1054215133,1.3171 - 0.2929*x_1,"[0.13066960871219635, 0.08464369922876358, 0.0...",...,0.123924,0.957962,"[0.0447706, 2.1001563, 2.4137616, 0.69039524, ...",1.966265,1.131818,0.014595,"[2.0970429182052612, 2.0925717962338006, 2.088...",1.861858,1.067946,0.036619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,D2N2,0.00,nogap,100,"[2, 4, 4, 4, 1]",1,3,142085429,nan,"[11.7371826171875, 8.620177268981934, 8.593214...",...,15.678324,-3.332015,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN
76,D2N2,0.05,nogap,100,"[2, 5, 1]",1,3,142085429,-0.525*x_1 + 79.2664*x_2 + 265.92*exp(-0.0071*...,"[3.4832518100738525, 2.26466965675354, 2.14596...",...,3.678175,0.749739,"[-5.762806, -8.522202, 0.0916543, -3.5363598, ...",441.371338,11.074652,0.488866,"[0.0695855800192362, 0.2151867973180046, 0.366...",49.827419,5.618048,0.768348
77,D2N2,0.05,nogap,100,"[2, 5, 1]",1,3,142085429,-2.6389*x_1 + 121.5056*x_2 - 635.3733 + 630.98...,"[3.4832284450531006, 2.2660255432128906, 2.096...",...,9.917166,-0.667089,"[-5.6256313, -8.967255, 0.13431787, -4.5419044...",444.608643,11.234545,0.485117,"[3.581029671816111, 3.6692273246617546, 3.7639...",46.510169,5.687591,0.783770
78,D2N2,0.05,nogap,100,"[2, 4, 4, 4, 1]",1,3,142085429,-15.5075*x_1 + 185.6212*x_2 - 0.1575*(2.8755*x...,"[16.59711456298828, 12.283892631530762, 7.9586...",...,6.354827,0.467614,"[1.1118567, 0.6624751, 1.8204331, 2.0332298, -...",427.444397,9.188666,0.504995,"[-9.278253775449514, -9.075767792525745, -8.86...",19.758152,3.458187,0.908142


In [ ]:

# Ваш исходный код до обработки
folder = Path('./results')
files = [f.name for f in folder.iterdir() if f.is_file()]
print(files)

data_list = []
for file in files:
    if file == '.gap':
        continue
    data_list.append(pd.read_csv('./results/' + file, sep=';'))

data = pd.concat(data_list, axis=0, ignore_index=True)

# Обработка столбцов loss_cols с сохранением NaN
for loss in loss_cols:
    def convert_cell(x):
        # Если значение уже NaN (или None) — оставляем как есть
        if pd.isna(x):
            return x  # или return np.nan — эквивалентно
        
        # Если это строка — пытаемся разобрать как Python-литерал
        if isinstance(x, str):
            try:
                return ast.literal_eval(x.strip())
            except (SyntaxError, ValueError):
                # Если строка невалидна — возвращаем NaN
                return np.nan
        # Если не строка и не NaN — возвращаем как есть (на всякий случай)
        return x

    data[loss] = data[loss].apply(convert_cell)

# Проверка результата
print(data[loss_cols].head())


['.gap', 'results_D1N1.csv', 'results_D1N2.csv', 'results_D1N3.csv', 'results_D2N1.csv', 'results_D2N2.csv']
                     loss_train_history_before_prune  \
0  [0.09865006059408188, 0.018926279619336128, 0....   
1  [0.05829606577754021, 0.018773939460515976, 0....   
2  [0.0424366295337677, 0.012504816055297852, 0.0...   
3  [0.14159362018108368, 0.014349247328937054, 0....   
4  [0.11219257116317749, 0.029523508623242378, 0....   

                      loss_test_history_before_prune  \
0  [0.1627325415611267, 0.057213570922613144, 0.0...   
1  [0.0644860714673996, 0.035981398075819016, 0.0...   
2  [0.04931975528597832, 0.014539098367094994, 0....   
3  [0.13338617980480194, 0.015438261441886425, 0....   
4  [0.15769392251968384, 0.04224418103694916, 0.0...   

                            reg_history_before_prune  \
0  [13.233120918273926, 12.323894500732422, 10.66...   
1  [14.031579971313477, 13.582538604736328, 13.02...   
2  [14.90032958984375, 14.449875831604004, 13.533

In [8]:
metrics = []
for stage in stages:
    metrics += [
        f'test_mse_{stage}',
        f'test_mae_{stage}',
        f'test_r2_{stage}',
        f'true_mse_{stage}',
        f'true_mae_{stage}',
        f'true_r2_{stage}',
    ]
core_cols = [
    'func_id', 'noise_level', 'gap_str', 'n_samples',
    'width', 'k', 'grid'
]

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 53 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   func_id                            40 non-null     object 
 1   noise_level                        40 non-null     float64
 2   gap_str                            40 non-null     object 
 3   n_samples                          40 non-null     int64  
 4   width                              40 non-null     object 
 5   k                                  40 non-null     int64  
 6   grid                               40 non-null     int64  
 7   seed                               40 non-null     int64  
 8   formula                            40 non-null     object 
 9   loss_train_history_before_prune    40 non-null     object 
 10  loss_test_history_before_prune     40 non-null     object 
 11  reg_history_before_prune           40 non-null     object 
 

In [14]:
data[metrics] = data[metrics].astype('float')

In [15]:
result = data[core_cols+metrics].groupby(core_cols)[metrics].agg('mean').reset_index()
result

,func_id,noise_level,gap_str,n_samples,width,k,grid,test_mse_before_prune,test_mae_before_prune,test_r2_before_prune,...,test_r2_after_refine,true_mse_after_refine,true_mae_after_refine,true_r2_after_refine,test_mse_after_symbolic,test_mae_after_symbolic,test_r2_after_symbolic,true_mse_after_symbolic,true_mae_after_symbolic,true_r2_after_symbolic
0,D1N1,0.00,nogap,100,"[1, 4, 4, 4, 1]",1,3,0.195273,0.098824,0.920340,...,0.964999,0.208883,0.182307,0.891918,0.222263,0.334637,0.854405,0.285305,0.377261,0.852374
1,D1N1,0.00,nogap,100,"[1, 5, 1]",1,3,0.003794,0.041878,0.997415,...,0.375817,0.451797,0.256101,0.766227,2.167762,1.232853,-0.190897,1.916812,1.078963,0.008184
2,D1N1,0.05,nogap,100,"[1, 4, 4, 4, 1]",1,3,0.012781,0.071419,0.994551,...,0.962426,0.102206,0.135547,0.947116,0.464556,0.348675,0.785611,0.476456,0.316313,0.753467
3,D1N1,0.05,nogap,100,"[1, 5, 1]",1,3,0.009511,0.073294,0.995109,...,0.956000,0.229825,0.181205,0.881082,1.156025,0.818528,0.454653,1.129373,0.805329,0.415628
4,D1N2,0.00,nogap,100,"[1, 4, 4, 4, 1]",1,3,0.000509,0.017338,0.995225,...,0.984660,0.002273,0.024615,0.980882,0.062469,0.165851,0.400543,0.061706,0.160484,0.481074
5,D1N2,0.00,nogap,100,"[1, 5, 1]",1,3,0.000184,0.010133,0.998135,...,0.988043,0.013341,0.028928,0.887810,0.114109,0.296808,-0.094013,0.122044,0.309123,-0.026343
6,D1N2,0.05,nogap,100,"[1, 4, 4, 4, 1]",1,3,0.001897,0.038049,0.981918,...,0.934763,0.004417,0.041013,0.962852,0.054307,0.164621,0.482471,0.060060,0.165375,0.494921
7,D1N2,0.05,nogap,100,"[1, 5, 1]",1,3,0.001803,0.034307,0.982820,...,0.922621,0.002382,0.027502,0.979970,0.106299,0.291335,-0.013000,0.119281,0.307859,-0.003107
8,D1N3,0.00,nogap,100,"[1, 4, 4, 4, 1]",1,3,0.102295,0.140550,0.710985,...,0.049847,0.165212,0.167951,0.699165,0.404685,0.499035,-0.143364,0.449777,0.489255,0.181001
9,D1N3,0.00,nogap,100,"[1, 5, 1]",1,3,0.310754,0.261436,0.122022,...,-1.318178,0.249982,0.248846,0.544807,0.682627,0.664479,-0.928638,0.565040,0.607155,-0.028882


In [16]:
data['test_mse_before_prune'][0]

np.float64(0.0008427302236668766)

In [11]:
data

,func_id,noise_level,gap_str,n_samples,width,k,grid,seed,formula,loss_train_history_before_prune,...,true_mae_after_refine,true_r2_after_refine,test_residuals_after_symbolic,test_mse_after_symbolic,test_mae_after_symbolic,test_r2_after_symbolic,true_residuals_after_symbolic,true_mse_after_symbolic,true_mae_after_symbolic,true_r2_after_symbolic
0,D1N1,0.00,nogap,100,"[1, 5, 1]",1,3,600076812,1.4882 - 0.3372*x_1,"[0.22302645444869995, 0.0923803374171257, 0.06...",...,0.147888,0.896281,"[-2.4055583, 1.7071576, 1.7983644, -1.7691612,...",2.020545,1.189563,0.109174,"[1.837373971939087, 1.8332574976040399, 1.8292...",1.892972,1.078047,0.020519
1,D1N1,0.00,nogap,100,"[1, 5, 1]",1,3,1002779995,nan,"[0.13159415125846863, 0.08717337995767593, 0.0...",...,0.151268,0.874985,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN
2,D1N1,0.00,nogap,100,"[1, 4, 4, 4, 1]",1,3,785117571,nan,"[0.667601466178894, 0.3491753041744232, 0.2004...",...,0.139006,0.944912,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN
3,D1N1,0.00,nogap,100,"[1, 4, 4, 4, 1]",1,3,41416371,-0.4058*x_1 - 5.504*sin(1.1431*x_1 - 4.4575) +...,"[0.5994330644607544, 0.15937338769435883, 0.13...",...,0.229275,0.827259,"[-0.0015078187, -0.0035214424, 0.0023185015, 0...",0.000008,0.002594,0.999997,"[0.007747650146484375, 0.006915391738561372, 0...",0.000008,0.002047,0.999996
4,D1N1,0.05,nogap,100,"[1, 5, 1]",1,3,1054215133,1.3171 - 0.2929*x_1,"[0.13066960871219635, 0.08464369922876358, 0.0...",...,0.123924,0.957962,"[0.0447706, 2.1001563, 2.4137616, 0.69039524, ...",1.966265,1.131818,0.014595,"[2.0970429182052612, 2.0925717962338006, 2.088...",1.861858,1.067946,0.036619
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,D2N2,0.00,nogap,100,"[2, 4, 4, 4, 1]",1,3,142085429,nan,"[11.7371826171875, 8.620177268981934, 8.593214...",...,15.678324,-3.332015,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",NaN,NaN,NaN
76,D2N2,0.05,nogap,100,"[2, 5, 1]",1,3,142085429,-0.525*x_1 + 79.2664*x_2 + 265.92*exp(-0.0071*...,"[3.4832518100738525, 2.26466965675354, 2.14596...",...,3.678175,0.749739,"[-5.762806, -8.522202, 0.0916543, -3.5363598, ...",441.371338,11.074652,0.488866,"[0.0695855800192362, 0.2151867973180046, 0.366...",49.827419,5.618048,0.768348
77,D2N2,0.05,nogap,100,"[2, 5, 1]",1,3,142085429,-2.6389*x_1 + 121.5056*x_2 - 635.3733 + 630.98...,"[3.4832284450531006, 2.2660255432128906, 2.096...",...,9.917166,-0.667089,"[-5.6256313, -8.967255, 0.13431787, -4.5419044...",444.608643,11.234545,0.485117,"[3.581029671816111, 3.6692273246617546, 3.7639...",46.510169,5.687591,0.783770
78,D2N2,0.05,nogap,100,"[2, 4, 4, 4, 1]",1,3,142085429,-15.5075*x_1 + 185.6212*x_2 - 0.1575*(2.8755*x...,"[16.59711456298828, 12.283892631530762, 7.9586...",...,6.354827,0.467614,"[1.1118567, 0.6624751, 1.8204331, 2.0332298, -...",427.444397,9.188666,0.504995,"[-9.278253775449514, -9.075767792525745, -8.86...",19.758152,3.458187,0.908142


In [12]:
data['loss_train_history_before_prune'][0]

[0.22302645444869995,
 0.0923803374171257,
 0.06450283527374268,
 0.04896692559123039,
 0.044107064604759216,
 0.04276125505566597,
 0.040788084268569946,
 0.037650398910045624,
 0.036894507706165314,
 0.0357574000954628,
 0.0348334014415741,
 0.032881878316402435,
 0.03167275711894035,
 0.029017817229032516,
 0.028873462229967117,
 0.027535073459148407,
 0.02664584293961525,
 0.026094377040863037,
 0.02507537230849266,
 0.024532457813620567]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Цикл по группам данных
for i in range(len(data)):
    # Создаём фигуру с 3 подграфиками (горизонтально)
    fig, axes = plt.subplots(1, 4, figsize=(15, 5))
    fig.suptitle(f'Анализ: {str(data['func_id'][i]) + ' noise ' + str(data['noise_level'][i]) + str(data['gap_str'][i]) + str(data['n_samples'][i]) + str(data['width'][i]) + str(data['k'][i]) +	str(data['grid'][i])}', fontsize=16, y=0.98)  # общий заголовок

    # 1. MSE (значения по порядку)
    axes[0].plot(data['loss_train_history_before_prune'][i], color='blue', alpha=0.7, linewidth=1.5)
    axes[0].set_title('loss_train_history_before_prune')
    axes[0].grid(True, alpha=0.3)

    # 2. prune (распределение)
    axes[1].plot(data['loss_train_history_after_prune'][i], color='blue', alpha=0.7, linewidth=1.5)
    axes[1].set_title('loss_train_history_after_prune')
    axes[1].grid(True, alpha=0.3)

    # 3. refine (распределение)
    axes[2].plot(data['loss_train_history_after_refine'][i], color='blue', alpha=0.7, linewidth=1.5)
    axes[2].set_title('loss_train_history_after_refine')
    axes[2].grid(True, alpha=0.3)

    # 4. symbolic (распределение)
    axes[3].plot(data['loss_train_history_after_symbolic'][i], color='blue', alpha=0.7, linewidth=1.5)
    axes[3].set_title('loss_train_history_after_symbolic')
    axes[3].grid(True, alpha=0.3)

    # Общая настройка
    plt.tight_layout()
    plt.show()

